<a href="https://colab.research.google.com/github/JunHeeJang913/Google_colab/blob/main/statQuest_decisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd    #to load and manipulate data and for One - hot encoding
import numpy as np   # to calculate the mean and std dev
import matplotlib.pyplot as plt # to draw graph
from sklearn.tree import DecisionTreeClassifier # to build a classification tree
from sklearn.tree import plot_tree # to draw tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix